In [21]:
import numpy as np
import uproot
from copy import deepcopy
import os
import glob
import plotly.graph_objs as go
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, Output
from IPython.display import display, clear_output
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.optimize import fmin
import matplotlib.pyplot as plt
import pandas as pd
import particle


In [22]:
df = uproot.open("../build/test.root:Hits")["pixelHits"].arrays(library="pd")
geom = uproot.open("../build/test.root:geometry").arrays(library="np")

In [23]:
print(df.iloc[0])
row = df.iloc[0]
# long_df = pd.DataFrame({
#     "x": row["hit_rowID"],
#     "y": row["hit_colID"],
#     "z": row["hit_layerID"],
#     "pdg": row["hit_pdgc"]
# })

long_df = pd.DataFrame({
    "x": [geom['pixel_Xpos'][0][int(i)] for i in row["hit_rowID"]],
    "y": [geom['pixel_Ypos'][0][int(i)] for i in row["hit_colID"]],
    "z": [geom['pixel_Zpos'][0][int(i)] for i in row["hit_layerID"]],
    "pdg": row["hit_pdgc"],
    "isPrimary": row["hit_fromPrimaryLepton"]
})

print("Particle species: ", set(long_df["pdg"]))

event_id                                                                 0
hit_rowID                [4691.0, 4691.0, 4692.0, 4692.0, 4794.0, 4795....
hit_colID                [4829.0, 4830.0, 4830.0, 4831.0, 4749.0, 4749....
hit_layerID              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
hit_pdgc                 [-11, -11, -11, -11, -11, -11, -11, -11, -11, ...
hit_trackID              [925, 925, 925, 925, 925, 925, 925, 925, 925, ...
hit_parentID             [850, 850, 850, 850, 850, 850, 850, 850, 850, ...
hit_px                   [-0.8943461, -0.93620265, -1.0154259, -1.08909...
hit_py                   [-1.4940065, -1.5471224, -1.4895365, -1.443535...
hit_pz                   [1.9246958, 1.8637596, 1.8814597, 1.877741, -1...
hit_energy               [2.6452758, 2.646655, 2.6553378, 2.6564927, 2....
hit_charge               [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
hit_fromPrimaryLepton    [False, False, False, False, False, False, Fal...
Name: 0, dtype: object
Pa

In [24]:
def pdg_to_particle(pdg):
    if abs(pdg) == 11:
        return "electron"
    if abs(pdg) == 13:
        return "muon"
    if abs(pdg) == 15:
        return "tau"
    return "other"

long_df["particle"] = long_df["pdg"].apply(pdg_to_particle)


In [25]:
def pdg_to_name(pdg):
    return particle.Particle.from_pdgid(pdg).name

long_df["name"] = long_df["pdg"].apply(pdg_to_name)


In [26]:
def isPrim_to_particle(isPrim):
    if isPrim > 0:
        return "Primary"
    else:
        return "Secondary"

long_df["primary"] = long_df["isPrimary"].apply(isPrim_to_particle)

In [27]:
import plotly.express as px

fig = px.scatter_3d(
    long_df,
    x="x",
    y="y",
    z="z",
    color="particle",
    color_discrete_map={
        "electron": "blue",
        "muon": "orange",
        "tau": "purple",
        "other": "gray"
    },
    opacity=0.8,
    hover_name="name"
)

fig.update_traces(marker=dict(size=1))
fig.show()


In [28]:
import plotly.express as px

fig = px.scatter_3d(
    long_df,
    x="x",
    y="y",
    z="z",
    color="primary",
    color_discrete_map={
        "Primary": "orange",
        "Secondary": "gray"
    },
    opacity=0.8,
    hover_name="name"

)

fig.update_traces(marker=dict(size=1))
fig.show()
